# Проект по SQL

<br> Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 
<br> Ваша первая задача как аналитика — проанализировать базу данных.

## Описание данных

<br> __Таблица books__
Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

<br> __Таблица authors__
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

<br> __Таблица publishers__
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

<br> __Таблица ratings__
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

<br> __Таблица reviews__
Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

## Цель исследования

Проанализировать базу данных для формирования ценностного предложения для нового продукта.

## Обзор данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Исследуем содержание таблиц

In [2]:
query = '''

SELECT *
FROM books
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = '''

SELECT *
FROM authors
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query = '''

SELECT *
FROM publishers
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
query = '''

SELECT *
FROM ratings
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = '''

SELECT *
FROM reviews
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Содержание таблиц совпадает с описанием в ТЗ, двигаемся дальше.

## Выполнение запросов

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'

'''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [8]:
query = '''
SELECT b.book_id,
       b.title,
       COUNT(DISTINCT review_id) AS review_cnt,
       ROUND(AVG(rating),2) AS rating_avg
FROM books AS b

LEFT JOIN reviews AS rv 
ON b.book_id = rv.book_id

LEFT JOIN ratings AS r
ON b.book_id = r.book_id

GROUP BY b.book_id,b.title
ORDER BY review_cnt DESC
         
         
'''   
pd.io.sql.read_sql(query, con = engine)

,book_id,title,review_cnt,rating_avg
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


Количество обзоров книг варьируется от 7 до нуля, а средняя оценка от 3,66 до 4.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [9]:
query = '''
WITH
--исключаем брошюры--
tab AS (
SELECT publisher_id,
       book_id
FROM books
WHERE num_pages > 50
)
SELECT tab.publisher_id,
       p.publisher
FROM tab

INNER JOIN publishers AS p
ON tab.publisher_id = p.publisher_id

GROUP BY tab.publisher_id,p.publisher
ORDER BY COUNT(tab.book_id) DESC
LIMIT 1

'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,212,Penguin Books


Издательство, которое выпустило наибольшее число книг толще 50 страниц это Penguin Books.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [12]:
query = '''
SELECT author,
       AVG(rating) AS rating_avg
FROM authors AS a
     JOIN books AS b 
     ON a.author_id = b.author_id
     JOIN ratings AS r 
     ON b.book_id = r.book_id

WHERE b.book_id IN (
                  --считаем книги с количеством оценок 50 и более --
                  SELECT book_id
                  FROM ratings 
                  GROUP BY book_id
                  HAVING COUNT(rating_id) >= 50
                  ORDER BY COUNT(rating_id) DESC)
GROUP BY author
ORDER BY rating_avg DESC
LIMIT 1

'''
pd.io.sql.read_sql(query, con = engine)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книги (4.29) это J.K. Rowling с Mary GrandPré.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [11]:
query = '''
SELECT AVG(review_cnt) AS avg_reviews_cnt
FROM (
       --считаем количество обзоров по каждому пользователю
       SELECT username, 
              COUNT(review_id) AS review_cnt
       FROM reviews 
       WHERE username IN (
                        --определяем пользователей, которые поставили больше 48 оценок
                          SELECT username
                          FROM ratings
                          GROUP BY username
                          HAVING COUNT(rating_id) > 48)
       GROUP BY  username
      ) AS review_count
      
'''
pd.io.sql.read_sql(query, con = engine)

,avg_reviews_cnt
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок равно 24.

## Общий вывод

<br> В данной работе мы с использованием языка запросов SQL и проанализировали базу данных крупного сервиса для чтения книг по подписке.

<br> По полученной информации можно сделать вывод, что:
- Мы обладаем информацией о 1000 книг. 
- Количество обзоров на данные книги варьируется от 7 до полного их осутствия. 
- Издательство, которое выпустило наибольшее число книг не являющихся брошюрами это Penguin Books.
- Автором с самой высокой средней оценкой книги в 4.29 балла является J.K. Rowling, написавшая серию книг о Гарри Поттере вместе с иллюстратором Mary GrandPré.
- Количество обзоров на книгу не зависит от среднего рейтинга, потмоу что у читателей разные вкусы и интересы.

<br> __Чтобы нашей компании быть на волне и в полной мере использовать потенциал крупного сервиса для чтения книг стоит сделать упор на самое попурялое издательство и пополнять библиотеку книжными бестселлерами.__